# 医療テキスト分析システム

このノートブックでは以下の機能を実行できます：
1. ダミーデータの生成
2. LLMサーバーの起動確認
3. テキスト分析の実行
4. 結果の確認と保存

In [ ]:
import sys
import os
import pandas as pd
import requests
from IPython.display import display

# srcディレクトリをパスに追加
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.data.data_generator import MedicalDataGenerator
from src.analyzer.excel_analyzer import ExcelAnalyzer

## 1. LLMサーバーの起動確認

**注意**: 別ターミナルで `python src/analyzer/llm_server.py` を実行してサーバーを起動しておく必要があります。

In [ ]:
def check_llm_server():
    try:
        response = requests.get("http://localhost:8000/docs")
        if response.status_code == 200:
            print("✅ LLMサーバーが正常に動作しています")
            return True
    except:
        print("❌ LLMサーバーに接続できません")
        print("別ターミナルで以下のコマンドを実行してください：")
        print("python src/analyzer/llm_server.py")
        return False

check_llm_server()

## 2. ダミーデータの生成

分析用のサンプルデータを生成します。

In [ ]:
# ジェネレーターの初期化
generator = MedicalDataGenerator()

# データの生成（患者数は調整可能）
df = generator.generate_patient_records(num_patients=10)

# 生成されたデータの確認
print("生成されたデータのサンプル:")
display(df.head())

# データの保存
output_file = "../data/sample_data.xlsx"
os.makedirs(os.path.dirname(output_file), exist_ok=True)
generator.save_to_excel(output_file, num_patients=10)

## 3. テキスト分析の実行

生成したデータに対して、各種分析を実行します。

In [ ]:
# アナライザーの初期化
analyzer = ExcelAnalyzer(
    file_path="../data/sample_data.xlsx",
    llm_server_url="http://localhost:8000",
    template_path="../templates/prompt_templates.json"
)

# ファイルの読み込み
if not analyzer.load_excel():
    print("❌ ファイルの読み込みに失敗しました")
else:
    print("✅ ファイルの読み込みが完了しました")
    analyzer.display_data_info()

In [ ]:
# 各種分析の実行
analysis_templates = [
    "cancer_diagnosis",   # がん診断名の抽出
    "cancer_stage",       # ステージ情報の抽出
    "important_dates",    # 重要日付の抽出
    "diagnostic_test",    # 診断検査の抽出
    "first_treatment",    # 初回治療情報の抽出
    "chemotherapy_info",  # 抗がん剤治療情報の抽出
    "surgery_type",       # 術式の抽出
    "special_notes"       # 特記事項の抽出
]

for template in analysis_templates:
    print(f"\n分析実行中: {template}")
    analyzer.analyze_with_template(template)

## 4. 分析結果の確認と保存

In [ ]:
# 分析結果の保存
output_path = "../data/sample_data_analyzed.xlsx"
analyzer.save_results(output_path)

# 結果の確認
results_df = pd.read_excel(output_path)
print("\n分析結果のプレビュー:")
display(results_df.head())

## 5. 特定の患者の詳細確認

In [ ]:
def display_patient_details(df, patient_id):
    patient_data = df[df['ID'] == patient_id].sort_values('day')
    print(f"患者ID {patient_id} の経過:")
    
    for _, row in patient_data.iterrows():
        print(f"\n日付: {row['day']}")
        print(f"記録: {row['text']}")
        
        # 分析結果列の表示
        analysis_columns = [col for col in df.columns if col.startswith('分析結果_')]
        for col in analysis_columns:
            if row[col] != 'N/A':
                print(f"{col}: {row[col]}")

# 最初の患者IDについて詳細を表示
first_patient_id = results_df['ID'].iloc[0]
display_patient_details(results_df, first_patient_id)